In [1]:
%run stdPackages.ipynb
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other'], 
        'variable2D': ['Generators_FuelMix'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories']}
db = dbFromWB(os.path.join(d['data'],'0_GlobalData.xlsx'), read)
readSets(db)

# The ```mBasic.mEmissionCap``` model

In [2]:
m = mBasic.mEmissionCap(db)
m.solve()

### **The model**

The model is an extension to the model [mBasic](M_mBasic.ipynb); the note "*Models for Energy Economics*" describes the extension in section 1.2.1. The extension is that a cap on $CO_2$ emissions from fuel consumption. Specifically, as also outlined for ```mBasic```, fuel consumption and emissions can be computed as:
$$\begin{align}
    \text{FuelConsumption}_{BFt} =& \sum_{id}\mu_{BFt}^{id}\cdot E_{id} \\ 
    \text{Emissions}_m =& \sum_{BFt}\text{FuelConsumption}_{BFt}\cdot \text{EmissionIntensity}_{BFt,m},
\end{align}$$
where $\mu_{BFt}^{id}$ is the fuel intensity for plant $id$. The emission cap is thus included as the inequality constraint that:
$$\begin{align}
    \sum_{BFt} \sum_{id}\mu_{BFt}^{id}\cdot E_{id} \cdot \text{EmissionIntensity}_{BFt,CO2} \leq \text{CO2Cap}
\end{align}$$

### **Augmented-form LP**

To solve the model, the algorithm in ```scipy.optimize.linprog``` requires that we specify the problem in the 'augmented'-form that specifically looks as follows:
$$\begin{align} \tag{2}
    &\min_{x}\mbox{ }c^T\cdot x \\ 
    &A_{ub}\times x \leq b_{ub} \\ 
    &A_{eq}\times x  = b_{eq} \\ 
    &l\leq x\leq u,
\end{align}$$
where 
* $c,x,l,u$ are all vectors of the same length $N$, 
* $b_{eq},b_{ub}$ are vectors of lengths $N_{eq},N_{ub}$ respectively,
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

Extending the model with the emission cap is straightforward here, as the ```mBasic``` do not have any other inequality constraints and the extension does not warrant any new definition of variables. Thus, the augmented form becomes (note $A_{ub}$ is transposed here):
<b id='augmentedForm'>
    
$$\begin{align}\tag{3}
c = \begin{pmatrix} mc_{id1} \\ \vdots \\ mc_{idN}\end{pmatrix}, \qquad l = \begin{pmatrix} 0 \\ \vdots \\ 0 \end{pmatrix}, \qquad u = \begin{pmatrix} q_{id1} \\ \vdots \\ q_{idN}\end{pmatrix}, \qquad b_{eq} = L, \qquad A_{eq} = \begin{pmatrix} 1 & \cdots & 1 \end{pmatrix}, \qquad b_{ub} = \text{CO2Cap}, \qquad A_{ub} = \begin{pmatrix} \sum_{BFt}\mu_{BFt}^{id1} \cdot \text{EmissionIntensity}_{BFt,CO2} \\ \vdots \\ \sum_{BFt}\mu_{BFt}^{idN} \cdot \text{EmissionIntensity}_{BFt,CO2} \end{pmatrix}^T
\end{align}$$


### Adjustments to the code

Compared to ```mBasic``` the main adjustment is that there is an added ```block``` of the type ```ub```:

<img src="snippets/mBasic_EmissionCap_snippet.png" width="1300" height="400">

The specification of the constraint is in terms of syntax equivalent to the ```eq``` constraints:
* There is a constraint named ```emissionsCap``` ,
* with the parameter constraint $b_{ub}$ from ```self.db['CO2Cap']```.
* $A_{ub}$ coefficients for this constraint are on the variable ```'Generation'``` and specifically given by their emission intensity computed by the function ```plantEmissionIntensity``` (follows the formula in [equation (3)](#augmentedForm)).

In [3]:
mBasic.plantEmissionIntensity(m.db)

id
id1    0.413676
id2    0.104306
id3       0.066
id4    0.036637
id5    0.135125
id6         0.0
id7         0.0
id8         0.0
dtype: object